# Data Wrangling WeRateDogs

## Gathering

* **Twitter archive file** Manually download by clicking the link: 
* **Tweet Image prediction** Programtically download the file 
* **Tweet JSON file** Get from twitter using API. 

In [163]:
#Imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import requests 
import tweepy
import json
import time
import re

### 1 Twitter archive 

In [164]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

### 2 Tweet Image prediction

In [165]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
page = requests.get(url)
with open('image_predictions.tsv', mode = 'wb') as f: 
    f.write(page.content)
image_prediction = pd.read_csv('image_predictions.tsv', sep = '\t')

### 3 Tweet JSON file

In [166]:
consumer_key = 'IlUpuNICtvcgHR66TrBLN43k7'
consumer_secret = 'WZXOLRl7sBUpfIBeoukENQ5Z0ZcfPUOB6ouyxSrTYwNcRTXhSs'
access_token = '1307275021522530306-jmWn1Fm4mu7i431hpL4gnMsd3dTCfY'
access_secret = 'NtlNyDFUgIPFcTxlFDhEPzSeVRAv1vtQLm61dDIBPzFZ5'

In [167]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [168]:
#Create a list of the tweet ids from the image_prediction DataFrame. 
tweet_ids = list(image_prediction['tweet_id'])

In [169]:
#quick test to check if we can pull tweets correctly. 
tweet = api.get_status(tweet_ids[0], tweet_mode = 'extended')
print(tweet.full_text)

Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj


In [170]:
#start = time.time()
#with open('tweet_json.txt', 'w') as outfile: 
    #for tweet_id in tweet_ids: 
        #try:
            #tweet = api.get_status(tweet_id, tweet_mode = 'extended')
            #json.dump(tweet._json, outfile)
            #outfile.write('\n')
        #except Exception as e:
            #print(e)

#end = time.time()
#print('Process complete, time taken: {}s.'.format(end-start))

In [171]:
#Create pandas DataFrame object from the json file
tweet_df = pd.DataFrame(columns =['tweet_id', 'retweet_count', 'favorite_count', 'created_at'])
with open('tweet_json.txt') as f:
    for line in f:
        status = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        created_at = status['created_at']
        tweet_df = tweet_df.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count, created_at]], columns =['tweet_id', 'retweet_count', 'favorite_count', 'created_at']))
                                   

tweet_df = tweet_df.reset_index(drop=True)

## Cleaning

In this stage, the datasets will be cleaned and a master DataFrame will be created.

### Visual assessment

In [172]:
tweet_df.sample(10)

,tweet_id,retweet_count,favorite_count,created_at
822,693155686491000832,3138,7726,Fri Jan 29 19:36:08 +0000 2016
285,671147085991960577,201,625,Mon Nov 30 02:01:49 +0000 2015
197,669603084620980224,337,899,Wed Nov 25 19:46:30 +0000 2015
1156,733482008106668032,920,3060,Fri May 20 02:18:32 +0000 2016
342,672256522047614977,1440,2676,Thu Dec 03 03:30:19 +0000 2015
1003,708845821941387268,876,2911,Sun Mar 13 02:43:08 +0000 2016
1032,711694788429553666,17866,32054,Sun Mar 20 23:23:54 +0000 2016
1108,723912936180330496,1200,3791,Sat Apr 23 16:34:28 +0000 2016
1880,849051919805034497,8150,41118,Tue Apr 04 00:12:06 +0000 2017
967,706346369204748288,906,3375,Sun Mar 06 05:11:12 +0000 2016


In [173]:
image_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
747,687704180304273409,https://pbs.twimg.com/media/CYs3TKzUAAAF9A2.jpg,1,miniature_pinscher,0.956063,True,toy_terrier,0.012231,True,Chihuahua,0.005397,True
980,707315916783140866,https://pbs.twimg.com/media/CdDkEkHWwAAAeUJ.jpg,2,Bernese_mountain_dog,0.979235,True,Shetland_sheepdog,0.011037,True,Appenzeller,0.003971,True
1216,743545585370791937,https://pbs.twimg.com/media/ClGawiUWAAAgs0w.jpg,2,rapeseed,0.876875,False,standard_poodle,0.060584,True,Great_Pyrenees,0.033006,True
109,667885044254572545,https://pbs.twimg.com/media/CUTN5V4XAAAIa4R.jpg,1,malamute,0.088530,True,golden_retriever,0.087499,True,muzzle,0.075008,False
1458,777885040357281792,https://pbs.twimg.com/media/CsuaUH2WAAAWJh1.jpg,1,Afghan_hound,0.123529,True,basset,0.119682,True,Siberian_husky,0.108709,True
964,706166467411222528,https://pbs.twimg.com/media/CczOp_OWoAAo5zR.jpg,1,Samoyed,0.430418,True,kuvasz,0.279600,True,Great_Pyrenees,0.117480,True
203,669749430875258880,https://pbs.twimg.com/media/CUttjYtWcAAdPgI.jpg,1,washbasin,0.245794,False,toilet_seat,0.109420,False,paper_towel,0.105664,False
517,676440007570247681,https://pbs.twimg.com/media/CWMyl9EWUAAnZJ0.jpg,2,Maltese_dog,0.579472,True,toy_poodle,0.133446,True,Shih-Tzu,0.094397,True
1413,771004394259247104,https://pbs.twimg.com/media/CrMmVqyWcAIDCHI.jpg,1,home_theater,0.414338,False,iPod,0.052741,False,pop_bottle,0.048821,False
1517,787322443945877504,https://pbs.twimg.com/media/Cu0hlfwWYAEdnXO.jpg,1,seat_belt,0.747739,False,golden_retriever,0.105703,True,dingo,0.017257,False


In [174]:
twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1978,672984142909456390,NaN,NaN,2015-12-05 03:41:37 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Very happy pup here. Always smiling. Loves his little leaf. Carries it everywhere with him. 9/10 https://t.co/81BCQAyvcs,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672984142909456390/photo/1,9,10,None,None,None,None,None
1128,729823566028484608,NaN,NaN,2016-05-10 00:01:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Stefan. He's a downright remarkable pup. 13/10 https://t.co/Ebjt6Y4fMh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/729823566028484608/photo/1,13,10,Stefan,None,None,None,None
437,820078625395449857,NaN,NaN,2017-01-14 01:22:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've never wanted to go to a camp more in my entire life. 12/10 for all on board https://t.co/wJZlpGFEbD,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/820078625395449857/photo/1,https://twitter.com/dog_rates/status/820078625395449857/photo/1,https://twitter.com/dog_rates/status/820078625395449857/photo/1",12,10,None,None,None,None,None
296,837366284874571778,NaN,NaN,2017-03-02 18:17:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Lucy. She has a portrait of herself on her ear. Excellent for identification pupposes. 13/10 innovative af https://t.co/uNmxbL2lns,NaN,NaN,NaN,https://twitter.com/dog_rates/status/837366284874571778/photo/1,13,10,Lucy,None,None,None,None
1724,680085611152338944,NaN,NaN,2015-12-24 18:00:19 +0000,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",This is by far the most coordinated series of pictures I was sent. Downright impressive in every way. 12/10 for all https://t.co/etzLo3sdZE,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/680085611152338944/photo/1,https://twitter.com/dog_rates/status/680085611152338944/photo/1,https://twitter.com/dog_rates/status/680085611152338944/photo/1",12,10,by,None,None,None,None
1905,674606911342424069,6.744689e+17,4.196984e+09,2015-12-09 15:09:55 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",The 13/10 also takes into account this impeccable yard. Louis is great but the future dad in me can't ignore that luscious green grass,NaN,NaN,NaN,NaN,13,10,None,None,None,None,None
772,776477788987613185,NaN,NaN,2016-09-15 17:48:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Huck. He's addicted to caffeine. Hope it's not too latte to seek help. 11/10 stay strong pupper https://t.co/iJE3F0VozW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/776477788987613185/photo/1,https://twitter.com/dog_rates/status/776477788987613185/photo/1",11,10,Huck,None,None,pupper,None
1963,673352124999274496,NaN,NaN,2015-12-06 04:03:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",*lets out a tiny screech and then goes into complete cardiac arrest* 12/10 https://t.co/az5PLGzVNJ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673352124999274496/photo/1,12,10,None,None,None,None,None
2279,667405339315146752,NaN,NaN,2015-11-19 18:13:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Biden. Biden just tripped... 7/10 https://t.co/3Fm9PwLju1,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667405339315146752/photo/1,7,10,Biden,None,None,None,None
326,833826103416520705,NaN,NaN,2017-02-20 23:50:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Charlie. She asked u to change the chann

### Programmatic assessment

In [175]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2059 non-null   object
 1   retweet_count   2059 non-null   object
 2   favorite_count  2059 non-null   object
 3   created_at      2059 non-null   object
dtypes: object(4)
memory usage: 64.5+ KB


In [176]:
tweet_df.describe()

,tweet_id,retweet_count,favorite_count,created_at
count,2059,2059,2059,2059
unique,2059,1569,1819,2059
top,757729163776290825,407,0,Sun Dec 13 02:51:51 +0000 2015
freq,1,5,72,1


In [177]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [178]:
image_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [179]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [180]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [181]:
twitter_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [182]:
twitter_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

### Main issues

#### ```Twitter Archive```
##### Quality 
* Duplicate rows and unwanted columns pertaining to retweets.
* 2 numerators have a value of 0 (not dogs) and denominators have values over 10 (should not be possible).
* sources in html format. 
* In several columns, null values are non-null example: 'None' string. 
* Some names are invalid (EG: 'a', 'an').
* Some tweets aren't about dogs.

#### Tidiness

* 4 columns for the different dog stages, when only one is necessary.
* Create a master DataFrame where we merge the two others (will need to drop one timestamp column). 

#### ```tweet_df```
##### Quality
* tweet_id column is string when all the others are int.

#### ```image_prediction```
##### Quality
* Certain names in p1, p2, p3 columns don't start with capitals. 
* Certain predictions where not dogs EG: 'water_bottle'

##### Tidiness
* Missing a column which makes it clear what dog breed the neural network was the most confident about for a given image. 


In [183]:
#first, create copies of the DataFrames
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_df_clean = tweet_df.copy()

*Create a master DataFrame where we merge the two others*
> Merge ```image_predictions```,```tweet_df```and ```twitter_archive``` on ```tweet_id```.

In [184]:
twitter_archive_clean = twitter_archive_clean.merge(image_prediction_clean, left_on = 'tweet_id', right_on = 'tweet_id')

In [185]:
#First, change datatype of tweet_df_clean's tweet_id column from str to int. 
tweet_df_clean['tweet_id'] = tweet_df_clean['tweet_id'].astype(int)

#Then merge with the twitter_archive
twitter_archive_clean = twitter_archive_clean.merge(tweet_df_clean, left_on = 'tweet_id', right_on = 'tweet_id')

In [186]:
#Drop one timestamp column.
twitter_archive_clean.drop('created_at', axis=1, inplace=True)

> Check:

In [187]:
twitter_archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo',
       'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf',
       'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'retweet_count', 'favorite_count'],
      dtype='object')

```twitter_archive``` *source is in html format*
> Do some string manipulations to remove the html around the source. 

In [188]:
twitter_archive_clean['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2019
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       30
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      10
Name: source, dtype: int64

In [189]:
#define a function which extracts the source without html. 
def source_retainer(string):
    return string.split('"')[1]

In [190]:
twitter_archive_clean['source'] = twitter_archive_clean['source'].apply(source_retainer)

> Check:

In [191]:
twitter_archive_clean['source'].value_counts()

http://twitter.com/download/iphone              2019
http://twitter.com                                30
https://about.twitter.com/products/tweetdeck      10
Name: source, dtype: int64

```twitter_archive```: *some tweets aren't about dogs*
> Drop rows were we find words in the text like: 'only rate dogs', 'only send in dogs'..

In [192]:
no_dogs = 'only rate dogs|only send in dogs|only send dogs|stop sending|without a dog|not pet| w/out a dog'
twitter_archive_clean = twitter_archive_clean[~twitter_archive_clean['text'].str.contains(no_dogs)]

> Check:

In [193]:
twitter_archive_clean[twitter_archive_clean['text'].str.contains(no_dogs)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count


```twitter_archive```: *certain names are invalid (EG: 'a')*
> Try to extract names from the text. 

In [194]:
twitter_archive_clean[twitter_archive_clean['name'].str.islower()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
21,887517139158093824,NaN,NaN,2017-07-19 03:39:09 +0000,http://twitter.com/download/iphone,I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887517139158093824/video/1,...,0.130432,False,tow_truck,0.029175,False,shopping_cart,0.026321,False,10539,42785
50,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,http://twitter.com/download/iphone,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004380872706/video/1,...,0.281463,True,Angora,0.272066,False,Persian_cat,0.114854,False,14313,45654
284,828650029636317184,NaN,NaN,2017-02-06 17:02:17 +0000,http://twitter.com/download/iphone,"Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/828650029636317184/photo/1,https://twitter.com/dog_rates/status/828650029636317184/photo/1,https://twitter.com/dog_rates/status/828650029636317184/photo/1",...,0.649209,True,Chesapeake_Bay_retriever,0.198560,True,vizsla,0.056200,True,1327,9417
514,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,http://twitter.com/download/iphone,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1",...,0.226716,False,lighter,0.081941,False,switch,0.039009,False,3997,14357
654,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,http://twitter.com/download/iphone,RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,"https://twitter.com/dog_rates/status/741067306818797568/photo/1,https://twitter.com/dog_rates/status/741067306818797568/photo/1",...,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True,3026,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,http://twitter.com/download/iphone,This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well. 2/10 https://t.co/v5A4vzSDdc,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853826850816/photo/1,...,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False,759,1105
2053,666050758794694657,NaN,NaN,2015-11-16 00:30:50 +0000,http://twitter.com/download/iphone,This is a truly beautiful English Wilson Staff retriever. Has a nice phone. Privileged. 10/10 would trade lives with https://t.co/fvIbQfHjIe,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758794694657/photo/1,...,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True,51,122
2055,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,http://twitter.com/download/iphone,This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226329800704/photo/1,...,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,126,265
2056,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,http://twitter.com/download/iphone,Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/

In [195]:
twitter_archive_clean[twitter_archive_clean['name'].str.len()==1]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
50,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,http://twitter.com/download/iphone,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004380872706/video/1,...,0.281463,True,Angora,0.272066,False,Persian_cat,0.114854,False,14313,45654
514,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,http://twitter.com/download/iphone,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1,https://twitter.com/dog_rates/status/792913359805018113/photo/1",...,0.226716,False,lighter,0.081941,False,switch,0.039009,False,3997,14357
613,776201521193218049,NaN,NaN,2016-09-14 23:30:38 +0000,http://twitter.com/download/iphone,This is O'Malley. That is how he sleeps. Doesn't care what you think about it. 10/10 comfy af https://t.co/Pq150LeRaC,NaN,NaN,NaN,https://twitter.com/dog_rates/status/776201521193218049/photo/1,...,0.502228,True,black-and-tan_coonhound,0.154594,True,bloodhound,0.135176,True,2508,9573
809,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,http://twitter.com/download/iphone,This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/747885874273214464/photo/1,https://twitter.com/dog_rates/status/747885874273214464/photo/1",...,0.408450,True,Samoyed,0.141330,True,pug,0.083018,True,949,2889
811,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,http://twitter.com/download/iphone,Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857231626240/photo/1,...,0.768923,True,Chihuahua,0.029053,True,Shetland_sheepdog,0.029035,True,1139,4764
846,743222593470234624,NaN,NaN,2016-06-15 23:24:09 +0000,http://twitter.com/download/iphone,This is a very rare Great Alaskan Bush Pupper. Hard to stumble upon without spooking. 12/10 would pet passionately https://t.co/xOBKCdpzaa,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743222593470234624/photo/1,...,0.350629,True,soft-coated_wheaten_terrier,0.182782,True,golden_retriever,0.087662,True,1867,6073
1109,704859558691414016,NaN,NaN,2016-03-02 02:43:09 +0000,http://twitter.com/download/iphone,Here is a heartbreaking scene of an incredible pupper being laid to rest. 10/10 RIP pupper https://t.co/81mvJ0rGRu,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704859558691414016/photo/1,...,0.284428,True,teddy,0.156339,False,mitten,0.138915,False,526,2201
1118,704054845121142784,NaN,NaN,2016-02-28 21:25:30 +0000,http://twitter.com/download/iphone,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845121142784/photo/1,...,0.667939,True,kuvasz,0.228764,True,golden_retriever,0.043885,True,896,2852
1128,703079050210877440,NaN,NaN,2016-02-26 04:48:02 +0000,http://twitter.com/download/iphone,This is a Butternut Cumberfloof. It's not windy they just look like that. 11/10 back at it again with the red socks https://t.co/hMjzhdUHaW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/703079050210877440/photo/1,https://twitter.com/dog_rates/status/703079050210877440/photo/1",...,0.778503,True,Shetland_sheepdog,0.093834,True,Cardigan,0.060296,True,3003,7221
1134,702539513671897089,NaN,

In [196]:
#First one can be easily replaced
twitter_archive_clean['name'] = twitter_archive_clean['name'].str.replace('O', "O'Malley")

In [197]:
#We see some dogs have no names, we can replace the corresponding rows with 'None' string values
twitter_archive_clean[twitter_archive_clean['name'].str.islower()] = 'None'

>Check: 

In [199]:
twitter_archive_clean[twitter_archive_clean['name'].str.islower()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count


```twitter_archive```*Duplicate rows and unwanted columns pertaining to retweets*
> Drop these rows and columns. 

In [ ]:
#drop duplicates
twitter_archive_clean.drop_duplicates(inplace=True)

#drop retweets
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean['retweeted_status_id'])]

#drop other useless columns
#twitter_archive_clean.drop(['in_reply_to_status_id','in_reply_to_user_id'], axis =1, inplace=True)
twitter_archive_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)


> Check:

In [ ]:
twitter_archive_clean.columns

```twitter_archive``` : *4 columns for the different dog stages, when only one is necessary.*
> Create a new column dog_stage which melts these 4 stages. Then drop the columns.

In [ ]:
twitter_archive_clean.columns

In [ ]:
melt_cols = ['doggo', 'floofer', 'pupper', 'puppo']
kept_cols = [c for c in list(twitter_archive_clean.columns) if c not in melt_cols]

In [ ]:
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars = kept_cols, value_vars= melt_cols, var_name = 'stages', value_name ='dog_stage')
twitter_archive_clean.drop('stages', axis= 1, inplace= True)

> Check

In [ ]:
twitter_archive_clean['dog_stage'].value_counts()

```twitter_archive```: *In several columns, null values are non-null example: 'None' string.*
> Replace 'None' with null values. 

In [ ]:
twitter_archive_clean['dog_stage'].replace('None', np.nan, inplace = True)

In [ ]:
twitter_archive_clean.head()

```image_prediciton```: *p1,p2 and p3 columns have breeds that don't start with capitals*
> Capitalise all first letters

In [ ]:
twitter_archive_clean['p1'] = twitter_archive_clean['p1'].str.capitalize()
twitter_archive_clean['p2'] = twitter_archive_clean['p2'].str.capitalize()
twitter_archive_clean['p3'] = twitter_archive_clean['p3'].str.capitalize()

> Check:

In [ ]:
assert twitter_archive_clean['p1'].str.capitalize().all()

```image_prediciton```: *Certain predictions where not dogs EG: 'water_bottle'* 
> Remove rows where (p1,p2,p1)_dog = False. 

In [ ]:
twitter_archive_clean  = twitter_archive_clean[twitter_archive_clean['p1_dog'] == True]
twitter_archive_clean  = twitter_archive_clean[twitter_archive_clean['p2_dog'] == True]
twitter_archive_clean  = twitter_archive_clean[twitter_archive_clean['p3_dog'] == True]

> Check

In [ ]:
assert [twitter_archive_clean['p1_dog'] == True]
assert [twitter_archive_clean['p2_dog'] == True]
assert [twitter_archive_clean['p3_dog'] == True]

```twitter_archive```: *2 numerators have a value of 0 (not dogs) and 1 denominator has a value of 0 (should not be possible).*
>Drop these rows.

In [ ]:
#Remove numerator = 0
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['rating_numerator'] !=0]
#Remove denominator != 10
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['rating_denominator']==10]

> Check:

In [ ]:
assert [twitter_archive_clean['rating_numerator'] !=0]
assert [twitter_archive_clean['rating_denominator'] <=10]

```image_prediction```: *Missing a column which makes it clear what dog breed the neural network was confident about for a given image.*

> Create a new column which indicates the above. 

In [ ]:
predicted = []
for index, tweet in twitter_archive_clean.iterrows():
    if tweet['p1_conf'] >= tweet['p2_conf'] and tweet['p1_conf'] >= tweet['p3_conf']:
        predicted.append(tweet['p1'])
    elif tweet['p2_conf'] > tweet['p1_conf'] and tweet['p2_conf'] >= tweet['p3_conf']:
        predicted.append(tweet['p2'])
    else: 
        predicted.append(tweet['p3'])
    index +=1

In [ ]:
twitter_archive_clean['predicted'] = predicted

> Check: 

In [ ]:
twitter_archive_clean[['p1', 'p1_conf', 'p2', 'p2_conf',  'p3', 'p3_conf', 'predicted']]

## Exploratory Data Analysis
Give 3 insights and 1 visualisation.
### 1. The most identified breed by the classier

In [ ]:
twitter_archive_clean['predicted'].value_counts()

> The most commonly identified dog breed by the classifier was the Golden Retriever.

### 2. The predicted dog breed with the highest averge numerator

In [ ]:
twitter_archive_clean.groupby(['predicted'])['rating_numerator'].mean().sort_values(ascending = False)

In [ ]:
twitter_archive_clean.query('predicted == "Clumber"').shape[0]

> The Clumber breed has a whooping 27/10 average rating! (This could be because there are only 4 dogs in this dataset which have been identified as Clumbers).

## 3. Finding the overall trend in the number of tweets on this page since November 2015

In [ ]:
no_tweets = [twitter_archive_clean['timestamp'].value_counts()]

In [ ]:
my = []
for index, tweet in twitter_archive_clean.iterrows():
    mdy = tweet['timestamp'].split(' ')[0]
    my.append(mdy.split('-')[0:2])
    index +=1

twitter_archive_clean['my'] = my
len(my)

In [ ]:
twitter_archive_clean['my'] = twitter_archive_clean['my'].str.join(sep= '')

In [ ]:
tweet_count_df = pd.DataFrame()
tweet_count_df['my'] = twitter_archive_clean['my'].drop_duplicates()

In [ ]:
no_tweets = [int(i/10)for i in twitter_archive_clean.groupby('my').sum()['rating_denominator']]
tweet_count_df['no_tweets'] = no_tweets

In [ ]:
tweet_count_df.set_index('my', inplace = True)

In [ ]:
tweet_count_df.sort_index(inplace=True)

In [ ]:
tweet_count_df.reset_index(inplace=True)

In [ ]:
plt.figure(figsize = (30, 20))
plt.scatter(x = tweet_count_df['my'], y =tweet_count_df['no_tweets'])
plt.xlabel('date');
plt.ylabel('number of tweets')
plt.title('Number of tweets on the WeRateDogs page over time');

> Overall the number of tweets on the WeRateDogs page has increased steadily over a time, with a huge increase starting around March 2017. This means the page has gotten more popular over time. 